In [3]:
import soundfile as sf
from IPython.display import Audio
import numpy as np
import random 
import pandas as pd
import os
from os.path import join as pjoin
import tqdm
from acoustics.bands import third
import scipy.signal as sig
from IPython.display import Audio
# from masp import shoebox_room_sim as srs
from scipy.io import wavfile
#import mat73
import matplotlib.pyplot as plt

In [2]:
# import my modules:
import sys
sys.path.append('../src')
import joa_helpers as hlp
import importlib
importlib.reload(hlp);

In [3]:
np.random.seed(42)

In [4]:
# ---------- NOISE POOL ------------
# create df with paths to noise files 
noise_path='/home/ubuntu/Data/wham'
database="wham"
noise_pool = []
for split in ['tr', 'cv', 'tt']:
    split_path = pjoin(pjoin(noise_path, 'wham_noise'), split)
    for audio_path in os.listdir(split_path):
        noise_pool.append({'database_noise': database, 'noise_file_path': pjoin(split_path,audio_path)})

noise_pool=pd.DataFrame(noise_pool)
print(f"{len(noise_pool)=}")

noise_pool.to_csv("noise_pool.csv")
noise_pool.head(10)

len(noise_pool)=28000


,database_noise,noise_file_path
0,wham,/home/ubuntu/Data/wham/wham_noise/tr/012o030z_...
1,wham,/home/ubuntu/Data/wham/wham_noise/tr/01ra010k_...
2,wham,/home/ubuntu/Data/wham/wham_noise/tr/01zc0206_...
3,wham,/home/ubuntu/Data/wham/wham_noise/tr/20pa010i_...
4,wham,/home/ubuntu/Data/wham/wham_noise/tr/208a0105_...
5,wham,/home/ubuntu/Data/wham/wham_noise/tr/020c020l_...
6,wham,/home/ubuntu/Data/wham/wham_noise/tr/40ca010k_...
7,wham,/home/ubuntu/Data/wham/wham_noise/tr/408c020o_...
8,wham,/home/ubuntu/Data/wham/wham_noise/tr/01cc0211_...
9,wham,/home/ubuntu/Data/wham/wham_noise/tr/20eo010h_...


In [5]:
# ---------- SPEECH POOL ------------

# create df with paths to speech files
speech_dataset_path1 = '/home/ubuntu/Data/VCTK/wav48_silence_trimmed/'
speech_dataset_path2 = '/home/ubuntu/Data/LibriMix/LibriSpeech'
speech_dataset_path3 = '/home/ubuntu/Data/PTDB'

# initialize empty list of files:
speech_pool = []

# fill the list of files with filenames from vctk data base:
database="VCTK"
for root, dirs, files in os.walk(speech_dataset_path1):
    for file in files:
        if file.endswith('.flac'):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})

# fill the list of files with filenames from librispeech data base:
database="librispeech"
for split in ['train-clean-100', 'test-clean', 'dev-clean']:
    split_path = pjoin(pjoin(speech_dataset_path2, split))
    for root, dirs, files in os.walk(split_path):
        for file in files:
            if file.endswith('.flac'):
                speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})

# fill the list of files with filenames from ptdb data base:
database="PTDB"
for root, dirs, files in os.walk(speech_dataset_path3):
    for file in files:
        # make sure the correct speech files are used (MIC directory)
        if ("/MIC/" in root) & (file.endswith('.wav')):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})  

# shuffle order
random.shuffle(speech_pool)

# turn list to data frame 
speech_pool = pd.DataFrame(speech_pool)
print(f"{len(speech_pool)=}")

speech_pool.to_csv("speech_pool.csv")
speech_pool.head(10)

len(speech_pool)=126908


,database_speech,speech_file_path
0,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
1,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
2,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
3,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
4,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
5,librispeech,/home/ubuntu/Data/LibriMix/LibriSpeech/train-c...
6,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
7,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
8,librispeech,/home/ubuntu/Data/LibriMix/LibriSpeech/train-c...
9,librispeech,/home/ubuntu/Data/LibriMix/LibriSpeech/train-c...


In [6]:
# ---------- RIR POOL ------------
rir_path='/home/ubuntu/Data/synth_rirs_mono'
database="synth_rirs_mono"
# load df with rirs paths and stats:
rir_pool=pd.read_csv(pjoin(rir_path,"rir_info.csv"),index_col=0)
rir_pool["database_rir"]=database
print(f"{len(rir_pool)=}")
rir_pool.to_csv("rir_pool.csv")
rir_pool.head(10)

len(rir_pool)=10000


,room_x,room_y,room_z,volume,rt60_set,mic_pos_x,mic_pos_y,mic_pos_z,src_pos_x,src_pos_y,...,rt60_true,rt60_masp_stats,cd_masp_stats,mfp_masp_stats,rt30_meas,rt20_meas,edt_meas,c50_meas,ir_file_path,database_rir
0,3.087404,1.733557,2.544188,13.616981,0.094051,1.275112,0.636568,1.717796,1.375112,0.636568,...,0.094054,0.094054,0.678853,1.545836,0.087845,0.090524,0.222644,29.869119,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x309...,synth_rirs_mono
1,3.120417,1.627469,2.705770,13.740938,0.098842,1.533229,0.775045,1.112430,1.633229,0.775045,...,0.098846,0.098838,0.665230,1.533153,0.108747,0.107866,0.212617,25.080770,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x312...,synth_rirs_mono
2,3.061378,1.560564,3.018367,14.420179,0.096717,1.878979,0.702089,1.150549,1.978979,0.702089,...,0.096723,0.096712,0.688923,1.539943,0.096810,0.093025,0.235392,26.606241,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x306...,synth_rirs_mono
3,3.076184,1.544429,3.316970,15.758751,0.094286,1.419961,0.712446,1.134668,1.519961,0.712446,...,0.094291,0.094282,0.729411,1.569801,0.087765,0.089848,0.230418,27.731729,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x308...,synth_rirs_mono
4,3.015096,1.828398,3.086247,17.013851,0.100507,1.599783,0.966029,1.815395,1.699783,0.966029,...,0.100508,0.100506,0.734059,1.663050,0.099996,0.100491,0.220712,28.897935,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x302...,synth_rirs_mono
5,3.079932,1.645339,3.434983,17.406880,0.107771,1.097511,0.717791,1.034888,1.197511,0.717791,...,0.107776,0.107776,0.717009,1.634545,0.106246,0.109946,0.233664,22.603079,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x308...,synth_rirs_mono
6,3.478583,1.810856,2.799080,17.632004,0.096148,2.047192,0.964826,1.030445,2.147192,0.964826,...,0.096153,0.096144,0.764039,1.670902,0.093377,0.096400,0.196960,28.776429,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x348...,synth_rirs_mono
7,3.222587,1.953429,2.803276,17.646888,0.103874,1.995593,1.066677,1.723879,2.095593,1.066677,...,0.103878,0.103869,0.735387,1.696417,0.099488,0.097691,0.225181,26.747138,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x322...,synth_rirs_mono
8,3.094981,1.597018,3.713484,18.354788,0.092483,1.948308,0.915701,1.922755,2.048308,0.915701,...,0.092484,0.092484,0.794817,1.641282,0.088072,0.092394,0.212735,28.558985,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x309...,synth_rirs_mono
9,3.136293,1.779292,3.310702,18.474978,0.093152,1.900544,0.934894,1.239519,2.000544,0.934894,...,0.093153,0.093153,0.794544,1.690732,0.082575,0.086574,0.208506,32.160277,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x314...,synth_rirs_mono


In [7]:
rir_pool[rir_pool["rt60_set"]>1.5]

,room_x,room_y,room_z,volume,rt60_set,mic_pos_x,mic_pos_y,mic_pos_z,src_pos_x,src_pos_y,...,rt60_true,rt60_masp_stats,cd_masp_stats,mfp_masp_stats,rt30_meas,rt20_meas,edt_meas,c50_meas,ir_file_path,database_rir


In [9]:
# --------- CREATE METADATA FOR A DATASET BEING COMBINATIONS OF SPEECH, NOISE AND RIRS --------
from datetime import datetime
date_tag = datetime.now().strftime("%d-%m-%Y--%H-%M")
# create dataset with 20000 data points, which consists of random combinations of speech, noise and rirs
N_datapoints=100

# sample from noise pool: 
df_noise=noise_pool.sample(N_datapoints,replace=True)
# here plan ways to augment noise data set:
random_bool_values = [random.choice([1, -1]) for _ in range(len(df_noise))]
df_noise["aug_phase"]=random_bool_values

# sample from speech pool: 
df_speech=speech_pool.sample(N_datapoints)

# sample from rir pool: 
df_rir=rir_pool.sample(N_datapoints,replace=True)

# concatenate samples from speech, noise and rir pools
df_ds = pd.concat([df_speech.reset_index(drop=True), df_noise.reset_index(drop=True), df_rir.reset_index(drop=True)], axis=1,ignore_index=False)
df_ds = df_ds.reset_index(drop=True)

# randomize snr (only high snrs)
df_ds["snr"]= 200 #np.random.uniform(low=10, high=30, size=len(df_ds))

# Create test-train-val split:
df_ds.loc[0:N_datapoints*0.8,"split"]="train" # 80% training data
df_ds.loc[N_datapoints*0.8:N_datapoints*0.9,"split"]="test" # 10% testing data
df_ds.loc[N_datapoints*0.9:N_datapoints,"split"]="val" # 10% validation data

df_ds.to_csv(date_tag+"_data_set.csv")

In [12]:
# Change paths in the metadata from GuestXR computer: 

df = pd.read_csv("../dataset-metadata/nonoise2_guestxr2.csv",index_col=0)

df["speech_file_path"]=df["speech_file_path"].str.replace("/home/ubuntu/Data/","/media/ssd2/")
df["noise_file_path"]=df["noise_file_path"].str.replace("/home/ubuntu/Data/","/media/ssd2/")
df["ir_file_path"]=df["ir_file_path"].str.replace("/home/ubuntu/Data/","/media/ssd2/")

df.to_csv("../dataset-metadata/nonoise2_dacom.csv",index=False)

df.head(10)

,database_speech,speech_file_path,database_noise,noise_file_path,aug_phase,room_x,room_y,room_z,volume,rt60_set,...,cd_masp_stats,mfp_masp_stats,rt30_meas,rt20_meas,edt_meas,c50_meas,ir_file_path,database_rir,snr,split
0,librispeech,/media/ssd2/LibriMix/LibriSpeech/train-clean-1...,wham,/media/ssd2/wham/wham_noise/tr/01uo030s_1.9208...,1,20.356944,19.837310,4.267538,1723.347245,0.797306,...,2.623049,5.990526,1.358439,1.324450,0.465387,7.574104,/media/ssd2/synth_rirs_mono/monoRIR_x2036y1984...,synth_rirs_mono,200,train
1,librispeech,/media/ssd2/LibriMix/LibriSpeech/train-clean-1...,wham,/media/ssd2/wham/wham_noise/tr/20sa010i_1.2778...,1,22.813258,14.967837,4.589098,1567.016912,0.750975,...,2.577180,6.087320,1.221117,1.144523,0.493173,7.541716,/media/ssd2/synth_rirs_mono/monoRIR_x2281y1497...,synth_rirs_mono,200,train
2,VCTK,/media/ssd2/VCTK/wav48_silence_trimmed/p247/p2...,wham,/media/ssd2/wham/wham_noise/cv/027c0214_0.6624...,1,12.985778,11.598824,3.531631,531.933385,0.429641,...,1.985145,4.480505,0.601090,0.611684,0.378026,9.241051,/media/ssd2/synth_rirs_mono/monoRIR_x1299y1160...,synth_rirs_mono,200,train
3,librispeech,/media/ssd2/LibriMix/LibriSpeech/train-clean-1...,wham,/media/ssd2/wham/wham_noise/tr/01ec020r_2.4865...,1,26.994354,14.033851,2.990757,1133.002459,0.702691,...,2.265352,4.518093,1.467313,1.489414,0.887511,5.702895,/media/ssd2/synth_rirs_mono/monoRIR_x2699y1403...,synth_rirs_mono,200,train
4,VCTK,/media/ssd2/VCTK/wav48_silence_trimmed/p317/p3...,wham,/media/ssd2/wham/wham_noise/tr/01ic020a_0.4624...,-1,7.472313,7.210021,2.780418,149.796486,0.269339,...,1.330490,3.163649,0.331752,0.331296,0.350780,10.859154,/media/ssd2/synth_rirs_mono/monoRIR_x747y721z2...,synth_rirs_mono,200,train
5,librispeech,/media/ssd2/LibriMix/LibriSpeech/train-clean-1...,wham,/media/ssd2/wham/wham_noise/tr/019o0315_1.1245...,-1,27.257133,18.322527,3.036055,1516.265158,0.766824,...,2.508816,4.754659,1.652045,1.689787,1.056490,5.863899,/media/ssd2/synth_rirs_mono/monoRIR_x2726y1832...,synth_rirs_mono,200,train
6,VCTK,/media/ssd2/VCTK/wav48_silence_trimmed/p333/p3...,wham,/media/ssd2/wham/wham_noise/tr/205c0104_0.3999...,1,17.479244,16.493869,2.670974,770.042652,0.526206,...,2.158376,4.063103,1.044112,1.069466,0.652342,6.149875,/media/ssd2/synth_rirs_mono/monoRIR_x1748y1648...,synth_rirs_mono,200,train
7,VCTK,/media/ssd2/VCTK/wav48_silence_trimmed/p272/p2...,wham,/media/ssd2/wham/wham_noise/tr/20da010h_2.4019...,1,6.070682,5.523682,3.772643,126.506208,0.245936,...,1.279553,3.274228,0.239113,0.229818,0.343769,13.049105,/media/ssd2/synth_rirs_mono/monoRIR_x607y552z3...,synth_rirs_mono,200,train
8,librispeech,/media/ssd2/LibriMix/LibriSpeech/train-clean-1...,wham,/media/ssd2/wham/wham_noise/tr/028a010p_1.8049...,-1,21.091593,12.464904,4.685318,1231.792101,0.673197,...,2.413243,5.863896,1.030435,0.959109,0.396488,8.469015,/media/ssd2/synth_rirs_mono/monoRIR_x2109y1246...,synth_rirs_mono,200,train
9,VCTK,/media/ssd2/VCTK/wav48_silence_trimmed/p233/p2...,wham,/media/ssd2/wham/wham_noise/cv/025a010h_1.1923...,1,11.136938,10.264718,3.651445,417.424204,0.377974,...,1.874992,4.337675,0.479005,0.434067,0.342308,10.199988,/media/ssd2/synth_rirs_mono/monoRIR_x1114y1026...,synth_rirs_mono,200,train
